# Detekce mutací <i>Plasmodia falciparum</i> pomocí bioinformatických nástrojů v příkazové řádce
## Obsah hodiny
- Proč nás zajímají mutace plasmodia?
- Co je to příkazová řádka?
- Získání vstupních souborů
- Mapování sekvencí na referenční genom
- Hledání mutací 
- Obsahuje gen dhfr mutace?
- Vizualizace mutací

## Proč nás zajímají mutace plasmodia?
Malárie je život ohrožující onemocnění způsobené parazity rodu Plasmodium, které se na člověka přenášejí kousnutím infikovaných samic komárů Anopheles. Celosvětově nakazí každý rok ~250 milionů lidí, ~3 miliony na ni umírají.

Jedním z léků používaných k léčbě malárie je pyrimethamin. Bohužel u parazita Plasmodium falciparum se vyvinula rezistence vůči tomuto léku. Rezistenci způsobují mutace v genu dhfr (ID: PF3D7_0417200) (Cowman et al. 1988).

V tomto cvičení budeme analyzovat DNA sekvence z pacienta infikovaného malárií. Naším cílem je zjistit, zda parazit Plasmodium falciparum u tohoto pacienta nese mutace v genu dhfr, které by naznačovaly rezistenci na Pyrimethamin. Tato informace je důležitá pro volbu vhodné léčby.


## Co je to příkazová řádka?

<b>GUI = graphic user interface = grafické rozhraní</b>

Ovládání počítače pomocí interaktivních grafických prvků jako například okna, menu, klikací ikony

<b>CLI = command line interface = příkazová řádka (terminál) </b>

Ovládání počítače pomocí textových příkazů 

<details>
  <summary>Proč používat příkazovou řádku pro řešení bioinformatických úkolů?</summary>
   
+ flexibilita v kombinování existujících nástrojů a možnost zapojení vlastních skriptů
+ zpracování velkého množství souborů najednou
+ zpracování velkých souborů a náročných výpočtů na vzdálených serverech, kde GUI není dostupné
+ snadné zopakování analýzy kdykoli a kýmkoli  

</details>

<br/>
<b>Jak otevřít příkazovou řádku v Jupyter Lab</b>

V horním menu klikněte na <b>File</b> > <b>New</b> > <b>Terminal</b>

Vyzkoušejte zadat příkaz do příkazové řádky (potvrdit stisknutím <b>Enter<b>):

- `ls` - vypsat soubory v aktuálním adresáři
- `pwd` - vypsat cestu do aktuálního adresáře
- `date` - vypsat aktuální systémový čas

Nebo můžete spustit příkaz přímo v buňkách s kódem v tomto notebooku:  

In [ ]:
!ls 
# označte tuto buňku šipkami na klávesnici (nebo levým tlačítkem myši) a stiskněte Shift+Enter na klávesnici (nebo ikonu "Run this cell and advance" na horní liště)
# můžete taky obsah buňky přepsat - vyzkoušejte, jak se výstup příkazu změní, když u něj bude doplňkový parametr - jako například ls -la
# hashtag (#) signalizuje, že na řádku není zapsaný kód ke spuštění, ale komentář

<b>Poznámka:</b> `!` na začátku buňky říká Jupyter Labu "spusť kód jako příkaz v terminálu". Když do buňky napíšete kód bez vykřičníku: `print("ahoj")`, poběží jako python kód. Příkazová řádka a python používají jinou "gramatiku" takže je potřeba odlišit, jak se má příkaz spustit. My budeme dále posílat příkazy pouze do terminálu, takže pokud budete spouštět příkazy v buňkách JupyterLab Notebooku, všechny by měly začínat `!`. 

In [ ]:
print("ahoj pythone!")
!echo "ahoj příkazová řádko!"

## Získání vstupních souborů
potřebujeme: 
- sekvence DNA plasmodia z pacienta - krátká čtení
- referenční sekvenci plasmodia - DNA sekvence celého genomu 
- pozici genu <i>dhfr</i> (ID:PF3D7_0417200) v referenční sekvenci (tzv. genové anotace)

Všechny potřebné soubory pro toto cvičení (původně vytvořeny pro [tuto Galaxy lekci](https://training.galaxyproject.org/training-material/topics/introduction/tutorials/galaxy-intro-ngs-data-managment/tutorial.html#annotating-variants{target=_blank})) jsou dostupné v internetovém archivu, ze kterého si je můžeme stáhnout. Začátek příkazu (`wget`) určuje, jaký program se má spustit, pak následuje URL adresa souboru, který chceme stáhnout.

In [ ]:
!wget https://zenodo.org/records/15354240/files/GCF_000002765.6.fa.gz # referenční sekvence plasmodium falciparum - formát fasta
!wget https://zenodo.org/records/15354240/files/GCF_000002765.6_GCA_000002765.ncbiRefSeq.gtf.gz # genové anotace - formát gtf
!wget https://zenodo.org/records/15354240/files/ERR042228_F.fq.gz # soubor obsahující DNA čtení (DNA reads)  - formát fastq
!wget https://zenodo.org/records/15354240/files/ERR042228_R.fq.gz # soubor obsahující DNA čtení (DNA reads) - formát fastq


Stažené soubory mají příponu `.gz`, jsou tedy komprimované za účelem zmenšení jejich velikosti. Abychom s nimi mohli snadno pracovat, je nutné je nejdřív rozbalit pomocí `gunzip`.

<b>Poznámka:</b> `*` je takzvaný "žolíkový znak" (wildcard), který zastupuje libovolné znaky. Takže příkaz níže rozbalí všechny soubory uvnitř aktuální složky, které končí `.gz` a není potřeba vypisovat názvy jednotlivých souborů.


In [ ]:
!gunzip -f *.gz

<b>Poznámka:</b> Parametry připojené za příkazem modifikují jeho chování. Pokud se chcete podívat, co znamená parametr `-f`, podívejte se na nápovědu k příkazu pomocí `gunzip --help`.  

Nyní se můžeme podívat na obsah stažených souborů pomocí `head`, který nám vypíše prvních 10 řádků souboru.
Začneme s referenční sekvencí, která obsahuje celý genom plasmodia. 

In [ ]:
!head GCF_000002765.6.fa

Vidíme, že soubor obsahuje sekvenci nukleotidů (kombinace písmenek ATCG). První řádek ale obsahuje `>` a následně název sekvence. Genom plasmodia totiž obsahuje několik chromozomů, jejichž sekvence jsou v tomto souboru zapsané za sebou.
Názvy chromozomů a číslo řádku na kterém začínají si můžete vypsat, když pomocí příkazu `grep` vyhledáme v souboru řádky obsahující `>`.

In [ ]:
!grep -n ">" GCF_000002765.6.fa

Dále se můžeme podívat na soubory s DNA čteními, ve kterých budeme hledat mutace.

In [ ]:
!head ERR042228_F.fq

Tento soubor také zachycuje DNA sekvence, ale oproti předchozímu souboru obsahuje informace navíc. Na prvním řádku je opět název, na druhém samotná sekvence nukleotidů a na čtvrtém je kvalita čtení jednotlivých nukleotidů ve formě řady ASCII znaků. Začátky a konce DNA čtení často mají horší kvalitu, a tak je možné tuto informaci využít k filtrování čtení či částí čtení s nízkou kvalitou.

<b>Poznámka:</b> Vzorek má dva soubory DNA čtení (ERR042228_F.fq a ERR042228_R.fq), protože sekvenování bylo provedeno metodou, která přečte zhruba 100 nukleotidů z obou konců většího kusu DNA (~300 nukleotidů). Pro každý kus tak máme <b>F</b>orward a <b>R</b>everse sekvenci.  

Poslední soubor vyjadřuje pozice genů v referenčním genomu.

In [ ]:
!head GCF_000002765.6_GCA_000002765.ncbiRefSeq.gtf

Tento soubor je v postatě tabulka (oddělená tabulátorovou mezerou), která obsahuje název genu, název chromozomu, na kterém se daný gen nachází, přesnou numerickou pozici na daném chromozomu a další informace popisující vlastnosti genu, jako například jeho jednotlivé části (exony, introny, kódující sekvence, začátek transkriptu).
<details>
  <summary>Co obsahují jednotlivé sloupce tabulky?</summary>

- seqname - name of the chromosome or scaffold
- source - name of the program that generated this feature, or the data source (database or project name)
- feature - feature type name, e.g. Gene, Variation, Similarity
- start - Start position of the feature, with sequence numbering starting at 1.
- end - End position of the feature, with sequence numbering starting at 1.
- score - A floating point value.
- strand - defined as + (forward) or - (reverse).
- frame - One of '0', '1' or '2'. '0' indicates that the first base of the feature is the first base of a codon, '1' that the second base is the first base of a codon, and so on..
- attribute - A semicolon-separated list of tag-value pairs, providing additional information about each feature.
</details>

<br/>

Můžeme také v souboru vyhledat gen, který může obsahovat mutaci způsobující rezistenci.

In [ ]:
!grep "PF3D7_0417200" GCF_000002765.6_GCA_000002765.ncbiRefSeq.gtf

## Mapování sekvencí na referenční genom
Nejdřív si musíme nainstalovat nástroj pro mapování. Ten se nazývá BWA (Burrows-Wheeler Alignment) a funguje na základě hledání krátkých sekvencí v DNA readech, které se přesně shodují referencí. Aby bylo toto hledání rychlé, nástroj nejříve DNA sekvenci algoritmicky přetransformuje do podoby, ve které se tyto podobnosti hledají snáz - takzvaného indexu.

<b>Poznámka:</b> K instalaci nástrojů používáme jiný nástroj - mamba. Tento oblíbený software umožňuje snadno instalovat bioinformatické nástroje, a to i v případě, že nástroj ke své funkci potřebuje další nástroje od jiných tvůrců. Umožňuje tak vývojářům stavět na existujících balíčcích volně dostupných bioinformatických nástrojů.       

In [ ]:
!mamba install bioconda::bwa --yes

In [ ]:
!bwa index GCF_000002765.6.fa # nejdříve vytvoříme index z referenční sekvence
!bwa mem GCF_000002765.6.fa ERR042228_F.fq ERR042228_R.fq > ERR042228_mapping.sam # potom mapujeme DNA ready na referenční sekvenci, výsledek uložíme do souboru ERR042228_mapping.sam

## Hledání mutací
K hledání mutací (variant calling) v genomu použijeme nástroj LoFreq. V zásadě hledá rozdíly mezi referenční sekvencí a namapovanými ready, poté počítá kolik readů podporuje mutaci a kolik se shoduje s referencí. Bere také v potaz kvalitu readů a pokrytí daného regionu. Na základě statistických modelů se poté algoritmicky rozhodne, zda je rozdíl mezi ready a chybou sekvenování nebo skutečná mutace.

Nejdříve musíme nainstalovat potřebné nástroje a připravit soubor namapovaných readů do binárního formátu. Tento pak můžeme použít k detekci variant.

In [ ]:
!mamba install bioconda::samtools --yes # nainstalovat nástroj na konverzi formátu sam -> bam
!mamba install bioconda::lofreq --yes # nainstalovat nástroj na heldání mutací

In [ ]:
!samtools view -b ERR042228_mapping.sam > ERR042228_mapping.bam # konvertovat soubor do vhodného formátu
!samtools sort ERR042228_mapping.bam -o ERR042228_mapping.sorted.bam # seřadit namapovaná čtení podle jejich pozice v genomu pro efektivnějí 
!samtools faidx GCF_000002765.6.fa # vytvořit index s velikostí chromozomů v referenčním genomu pro usnadnění výpočtů a vizualizaci v IGV
!samtools index ERR042228_mapping.sorted.bam # vytvořit index namapovaných čtení pro usnadnění výpočtů a vizualizaci v IGV
!lofreq indelqual --dindel -f GCF_000002765.6.fa -o ERR042228_mapping.indelqual.bam ERR042228_mapping.sorted.bam # opravit chybně namapovaná čtení kolem inzercí a delecí
!lofreq call -f GCF_000002765.6.fa --call-indels --min-cov 10 --min-bq 20 --min-alt-bq 20 -q 20 -o ERR042228_variants.vcf ERR042228_mapping.indelqual.bam # najít mutace v namapovaných čteních

Nyní se můžeme podívat na nalezené mutace. Na začátku souboru je hlavička, která obsahuje informace o souboru. Můžeme se jí vyhnout pomocí příkazu `grep` a vypsat první řádky souboru, které neobsahují znak `#`.

In [ ]:
!grep -v "#" ERR042228_variants.vcf | head

Soubor obsahuje pozici mutace, který nukleotid se změnil a jak, jaká je kvalita a počet čtení v tomto místě.
<details>
  <summary>Co obsahují jednotlivé sloupce tabulky?</summary>

- CHROM	The name of the sequence (typically a chromosome) on which the variation is being called. This sequence is usually known as 'the reference sequence', i.e. the sequence against which the given sample varies.
- POS	The 1-based position of the variation on the given sequence.
- ID	The identifier of the variation, e.g. a dbSNP rs identifier, or if unknown a ".". Multiple identifiers should be separated by semi-colons without white-space.
- REF	The reference base (or bases in the case of an indel) at the given position on the given reference sequence.
- ALT	The list of alternative alleles at this position.
- QUAL	A quality score associated with the inference of the given alleles.
- FILTER	A flag indicating which of a given set of filters the variation has failed or PASS if all the filters were passed successfully.
- INFO    	An extensible list of key-value pairs (fields) describing the variation. See below for some common fields. Multiple fields are separated by semicolons with optional values in the format: <key>=<data>[,data].

</details>

## Obsahuje gen <i>dhfr</i> mutace?
Nyní známe pozici mutací, ale potřebujeme zjistit zda se některá z nich nachází v genu <i>dhfr</i>. To můžeme zjistit tak, že srovnáme pozice mutací a genů pomocí příkazu `bedtools intersect`.

In [ ]:
!mamba install bioconda::bedtools --yes # nainstalovat nástroj pro vyhledávání překrývajících se anotací

In [ ]:
!bedtools intersect -a ERR042228_variants.vcf -b GCF_000002765.6_GCA_000002765.ncbiRefSeq.gtf -wa -wb | grep "PF3D7_0417200" # najít mutace překrývající se s geny a vyhledat zda je některá v genu zájmu

Našli jsme mutaci, která se nachází v exonu <i>dhfr</i>. Tento kmen plasmodia tedy může být rezistentní k léčbě pyrimethaminem. Přitomnost mutace si můžeme ověřit v genomovém prohlížeči IGV. 

## Vizualizace mutací
Na webovou applikaci IGV můžeme nahrát referenční genom, genovou anotaci, a namapovaný ready, se kterými jsme pracovali. K tomu už zase budeme muset opustit příkazovou řádku a proklikat se grafickým rozhraním.
1. Otevřete webovou stránku https://igv.org/app/
2. V horním menu klikněta na <b>Genome</b> > <b>URL...</b>, mělo by se zobrazit okno pro vložení adres souborů
3. V JupyterLab klikněte pravým tlačítkem myši na soubor `GCF_000002765.6.fa` a vyberte <b>Copy Download Link</b>
4. V IGV aplikaci vložte odkaz do políčka <b>Genome URL</b>
5. Obdobně zkopírujte odkaz na `GCF_000002765.6.fa.fai` do políčka <b>Index URL</b> a potvrďte stisknutím <b>OK</b>
6. Poté otevřete okno na vložení anotací do IGV pomocí <b>Track</b> > <b>URL...</b>
7. Do políčka <b>Genome URL</b> vložte odkaz na genové anotace ze souboru `GCF_000002765.6_GCA_000002765.ncbiRefSeq.gtf`, <b>Index URL</b> nechte prázdné
8. Obdobně nahrajte namapované ready `ERR042228_mapping.sorted.bam`, do políčka <b>Index URL</b> zadejte odkaz na `ERR042228_mapping.sorted.bam.bai`
9. Do políčka <b>Locus Search</b> zadejte pozici mutace - `NC_004318.2:748410` 

## Reference
Použitý dataset byl vytvořen pro lekci NGS data logistics:

Anton Nekrutenko, Marius van den Beek, Dave Clements, Daniel Blankenberg, NGS data logistics (Galaxy Training Materials). [https://training.galaxyproject.org/training-material/topics/introduction/tutorials/galaxy-intro-ngs-data-managment/tutorial.html] Online; accessed Thu Feb 05 2026

Cowman, A. F., M. J. Morry, B. A. Biggs, G. A. Cross, and S. J. Foote, 1988 Amino acid changes linked to pyrimethamine resistance in the dihydrofolate reductase-thymidylate synthase gene of Plasmodium falciparum. Proc. Natl. Acad. Sci. U. S. A. 85: 10.1073/pnas.85.23.9109 [https://www.pnas.org/doi/abs/10.1073/pnas.85.23.9109]

